# Zadanie 1

### Import potrzebnych pibliotek

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from pathlib import Path
from sklearn.neighbors import KNeighborsRegressor

np.random.seed(42)

### Wczytanie danych

In [ ]:
datapath = Path("d:/m_w/wdum") / "data" / "lifesat"

oecd_bli = pd.read_csv(datapath / "oecd_bli.csv")
gdp_per_capita = pd.read_csv(datapath / "gdp_per_capita.csv")
# display(oecd_bli)
# display(gdp_per_capita)

### Obróbka gdp per capita

In [ ]:
gdp_per_capita_2020 = gdp_per_capita[gdp_per_capita["Year"]==2020]
gdp_per_capita_2020 = gdp_per_capita_2020.rename(
    columns={"GDP per capita, PPP (constant 2017 international $)": "GDP per capita $"})
gdp_per_capita_2020

In [ ]:
# oecd_bli["Indicator"].value_counts() // nie wiem co to tu robi

### Obróbka oecd bil

In [ ]:
oecd_bli = oecd_bli[oecd_bli["INEQUALITY"]=="TOT"]
oecd_bli = oecd_bli.pivot(index="Country", columns="Indicator", values="Value")
oecd_bli = oecd_bli.reset_index()
oecd_bli

### Obróbka i stworzenie danych dla krajów z połączenia oecd i gdp

In [ ]:
full_country_stats = oecd_bli.merge(gdp_per_capita_2020, left_on="Country", right_on="Entity", how="inner")
full_country_stats = full_country_stats[["Country","Life satisfaction", "GDP per capita $"]]
full_country_stats = full_country_stats.sort_values(by="GDP per capita $")
full_country_stats

### Wyeksportowanie danych dla krajów do csv

In [ ]:
full_country_stats.to_csv(datapath / "lifestat_full.csv", index=False)

### Zawężenie i eksport danych lifesat

In [ ]:
min_gdp = 23_500
max_gdp = 62_500

country_stats = full_country_stats[(full_country_stats["GDP per capita $"] >= min_gdp) & (full_country_stats["GDP per capita $"] <= max_gdp)]
country_stats.to_csv(datapath / "lifesat.csv", index=False)

### Wczytanie csv i obróbka lifesat

In [ ]:
lifesat = pd.read_csv(datapath / "lifesat.csv")

lifesat.rename(columns={"GDP per capita $": "PKB na mieszkanca (USD)"}, inplace=True)
lifesat.rename(columns={"Life satisfaction": "Satysfakcja z zycia"}, inplace=True)
lifesat.rename(columns={"Country": "Kraj"}, inplace=True)
lifesat

### Wizualizacja danych lifesat

In [ ]:
lifesat.plot(kind="scatter", grid=True, x="PKB na mieszkanca (USD)", y="Satysfakcja z zycia")
plt.show()

### Przygotowanie modelu regresji liniowej

In [ ]:
x = lifesat[["PKB na mieszkanca (USD)"]]
y = lifesat[["Satysfakcja z zycia"]]

model = LinearRegression() # wybranie modely

model.fit(x, y) # trenuje model regresji na naszych wartościach

theta_0 = model.intercept_[0] # zwraca thata_0 z funkcji satysfakcja z życia = theta_0 + theta_1 * x
theta_1 = model.coef_[0][0] # zwraca theta_1
print("theta_0: ", theta_0)
print("theta_1: ", theta_1)

### Wizualizacja modelu regresji liniowej

In [ ]:
lifesat.plot(kind="scatter", grid=True, x="PKB na mieszkanca (USD)", y="Satysfakcja z zycia") # wykres punktowy pkb i satysfakcji
plt.plot(x, theta_0+theta_1*x) # wykres liniowy zależnosci
plt.text(45000, 6.5, fr"$\theta_0 = {theta_0.round(2)}$", color="blue") # podpisanie thety 0
plt.text(45000, 6.3, fr"$\theta_1 = {theta_1 * 1e5:.2f}\cdot 10^{{-5}}$", color="blue") # podpisanie thety 1
plt.title("Zależnosć miedzy satysfakcją z życia, a PKB na mieszkańca")

plt.show()

### Stworzenie kolumny z pkb Cypru

In [ ]:
cyprus_pkb = (
    gdp_per_capita_2020[gdp_per_capita_2020["Entity"] == "Cyprus"][["GDP per capita $"]])
cyprus_pkb.rename(columns={"GDP per capita $": "PKB na mieszkanca (USD)"}, inplace=True)
cyprus_pkb

### Uzycie wytrenowanego modelu do obliczenia poziomu szczęścia na podstawie pkb cypru

In [ ]:
cyprus_predicted = model.predict(cyprus_pkb)
cyprus_predicted

### Zamiana modelu regresji liniowej na model najbliższych sąsiadów

In [ ]:
model_knn = KNeighborsRegressor(n_neighbors=3)

model_knn.fit(x,y)
model_knn.predict(cyprus_pkb)